# Preparation

## Imports

In [172]:
import pmp_functions_v5 as pmp
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Functions

In [173]:
def combine_strategy_results(dfs, weights):
    """
    Combine multiple strategy result DataFrames into a single meta-portfolio.

    Parameters
    ----------
    dfs : list of pd.DataFrame
        List of backtest result tables (each must contain ret_net, turnover, tcost, ret_rf, ret_bm, etc.)

    weights : list of float
        Strategy-level weights (must sum to 1.0)

    Returns
    -------
    final_results : pd.DataFrame
        Combined backtest results with aggregated performance and exposure.
    """

    import pandas as pd

    if abs(sum(weights) - 1.0) > 1e-8:
        raise ValueError("Strategy weights must sum to 1.0")

    # --------------------------------------------------------
    # 1) Align on common index (intersection)
    # --------------------------------------------------------
    common_index = dfs[0].index
    for df in dfs[1:]:
        common_index = common_index.intersection(df.index)

    dfs_aligned = [df.reindex(common_index) for df in dfs]

    # --------------------------------------------------------
    # 2) Combine performance metrics
    # --------------------------------------------------------
    ret_net_combined   = sum(w * df["ret_net"]   for w, df in zip(weights, dfs_aligned))
    ret_gross_combined = sum(w * df["ret_gross"] for w, df in zip(weights, dfs_aligned))
    ret_bm_combined    = sum(w * df["ret_bm"]    for w, df in zip(weights, dfs_aligned))
    turnover_combined  = sum(w * df["turnover"]  for w, df in zip(weights, dfs_aligned))
    tcost_combined     = sum(w * df["tcost"]     for w, df in zip(weights, dfs_aligned))
    ret_rf_combined    = sum(w * df["ret_rf"]    for w, df in zip(weights, dfs_aligned))

    # Performance-only DataFrame
    final_results = pd.DataFrame({
        "ret_net": ret_net_combined,
        "ret_gross": ret_gross_combined,
        "ret_bm": ret_bm_combined,
        "turnover": turnover_combined,
        "tcost": tcost_combined,
        "ret_rf": ret_rf_combined,
    })

    # --------------------------------------------------------
    # 3) Collect all weight columns across all strategies
    # --------------------------------------------------------
    region_cols = sorted({
        col for df in dfs_aligned for col in df.columns if col.startswith("w_")
    })

    # --------------------------------------------------------
    # 4) Ensure all strategies have all weight columns, fill missing with 0
    # --------------------------------------------------------
    for i, df in enumerate(dfs_aligned):
        for reg in region_cols:
            if reg not in df.columns:
                dfs_aligned[i][reg] = 0.0

    # --------------------------------------------------------
    # 5) Combine region weights using strategy weights
    # --------------------------------------------------------
    for reg in region_cols:
        final_results[reg] = sum(w * df[reg] for w, df in zip(weights, dfs_aligned))

    return final_results


In [174]:
def apply_fx_hedging(backtest_df, hedge_ratio, hedge_returns):
    """
    Adds FX hedging P&L on top of an existing backtest.

    - USD is treated as the base currency and will NOT be hedged.
    - The same hedge_ratio is used for long and short positions.
    """

    df = backtest_df.copy()

    # Extract region names from weight columns
    regions = [col.replace("w_", "") for col in df.columns if col.startswith("w_")]

    # Remove USD from hedging universe
    regions = [r for r in regions if r != "US"]

    # Align hedge data with Date index
    hedge_ratio = hedge_ratio.reindex(df.index)
    hedge_returns = hedge_returns.reindex(df.index)

    hedge_pnl = pd.Series(0.0, index=df.index)

    for region in regions:
        w_col = f"w_{region}"

        # Check hedge ratio availability (should match non-US regions)
        if region not in hedge_ratio.columns:
            print(f"⚠ Region {region} not in hedge_ratio — skipping.")
            continue

        # Check hedge return availability
        if region not in hedge_returns.columns:
            print(f"⚠ Region {region} not in hedge_returns — skipping.")
            continue

        # Use the same hedge ratio for long and short positions
        hedge_mult = hedge_ratio[region]

        # Hedge contribution
        contrib = df[w_col] * hedge_mult * hedge_returns[region]

        hedge_pnl += contrib

    # Add hedged P&L to ret_net
    df["ret_net"] = df["ret_net"] + hedge_pnl

    return df


In [175]:
def combine_macro_weights(dfs, weights):
    """
    Combine multiple macro-strategy weight tables into a single weighted average.

    Parameters
    ----------
    dfs : list of DataFrame
        Weight tables (w_AU, w_CH, ...)
    weights : list of float
        Strategy weights, must sum to 1.0

    Returns
    -------
    DataFrame
        Combined weighted sum of all weights
    """
    import pandas as pd
    import numpy as np

    # Align all to the same index
    common_index = dfs[0].index
    for df in dfs[1:]:
        common_index = common_index.intersection(df.index)

    dfs_aligned = [df.reindex(common_index) for df in dfs]

    # Identify region columns
    region_cols = [c for c in dfs_aligned[0].columns if c.startswith("w_")]

    # Weighted sum
    combined = pd.DataFrame(index=common_index)

    for col in region_cols:
        combined[col] = sum(w * df[col] for w, df in zip(weights, dfs_aligned))

    return combined


In [176]:
def combine_two_weight_tables(df1, df2, weight1=0.5, weight2=0.5):
    """
    Combine two weight tables (e.g., trend + macro) using weighted average.
    Missing regions in either table are treated as 0.
    """
    import pandas as pd

    # 1. Align indices
    idx = df1.index.union(df2.index)
    df1a = df1.reindex(idx)
    df2a = df2.reindex(idx)

    # 2. Align columns (regions)
    cols = sorted(set(df1a.columns).union(df2a.columns))
    df1a = df1a.reindex(columns=cols).fillna(0.0)
    df2a = df2a.reindex(columns=cols).fillna(0.0)

    # 3. Weighted combination
    combined = weight1 * df1a + weight2 * df2a

    return combined


## Data

### Strategies

In [177]:
# --- Fundamental Data ---
fundamental_equity = pd.read_csv("../Results/fundamental_equity.csv", index_col=0, parse_dates=True)
fundamental_equity.rename(columns={"w_GB": "w_UK"}, inplace=True)

# --- Macro Data ---
macroRS_equity = pd.read_csv("../Results/macroRS_equity.csv", index_col=0, parse_dates=True)
macroRS_bond = pd.read_csv("../Results/macroRS_bond.csv", index_col=0, parse_dates=True)
macroRS_rates = pd.read_csv("../Results/macroRS_rates.csv", index_col=0, parse_dates=True)
macroRS_fx = pd.read_csv("../Results/macroRS_fx.csv", index_col=0, parse_dates=True)

macroIT_equity = pd.read_csv("../Results/macroIT_equity.csv", index_col=0, parse_dates=True)
macroIT_bond = pd.read_csv("../Results/macroIT_bond.csv", index_col=0, parse_dates=True)
macroIT_rates = pd.read_csv("../Results/macroIT_rates.csv", index_col=0, parse_dates=True)
macroIT_fx = pd.read_csv("../Results/macroIT_fx.csv", index_col=0, parse_dates=True)

macroMP_equity = pd.read_csv("../Results/macroMP_equity.csv", index_col=0, parse_dates=True)
macroMP_bond = pd.read_csv("../Results/macroMP_bond.csv", index_col=0, parse_dates=True)
macroMP_rates = pd.read_csv("../Results/macroMP_rates.csv", index_col=0, parse_dates=True)
macroMP_fx = pd.read_csv("../Results/macroMP_fx.csv", index_col=0, parse_dates=True)    

macroBC_equity = pd.read_csv("../Results/macroBC_equity.csv", index_col=0, parse_dates=True)
macroBC_bond = pd.read_csv("../Results/macroBC_bond.csv", index_col=0, parse_dates=True)
macroBC_rates = pd.read_csv("../Results/macroBC_rates.csv", index_col=0, parse_dates=True)
macroBC_fx = pd.read_csv("../Results/macroBC_fx.csv", index_col=0, parse_dates=True)


# --- Trend Following Data ---
trend_equity = pd.read_csv("../Results/trend_equity.csv", index_col=0, parse_dates=True)
trend_bond = pd.read_csv("../Results/trend_bond.csv", index_col=0, parse_dates=True)
trend_rates = pd.read_csv("../Results/trend_rates.csv", index_col=0, parse_dates=True)
trend_fx = pd.read_csv("../Results/trend_fx.csv", index_col=0, parse_dates=True)

In [178]:
def extract_clean_weights(df):
    # 1) Nur die weight-Spalten nehmen
    weight_cols = [c for c in df.columns if c.startswith("w_")]
    
    # 2) Gewichtstabelle extrahieren
    w = df[weight_cols].copy()
    
    # 3) Prefix "w_" entfernen
    w.columns = [c.replace("w_", "") for c in weight_cols]
    
    return w

# Clean weights for both strategies
fundamental_equity = extract_clean_weights(fundamental_equity)

trend_equity = extract_clean_weights(trend_equity)
trend_bond = extract_clean_weights(trend_bond)
trend_rates = extract_clean_weights(trend_rates)
trend_fx = extract_clean_weights(trend_fx)

macroRS_equity = extract_clean_weights(macroRS_equity)
macroRS_bond = extract_clean_weights(macroRS_bond)
macroRS_rates = extract_clean_weights(macroRS_rates)
macroRS_fx = extract_clean_weights(macroRS_fx)
macroIT_equity = extract_clean_weights(macroIT_equity)
macroIT_bond = extract_clean_weights(macroIT_bond)
macroIT_rates = extract_clean_weights(macroIT_rates)
macroIT_fx = extract_clean_weights(macroIT_fx)
macroMP_equity = extract_clean_weights(macroMP_equity)
macroMP_bond = extract_clean_weights(macroMP_bond)
macroMP_rates = extract_clean_weights(macroMP_rates)
macroMP_fx = extract_clean_weights(macroMP_fx)
macroBC_equity = extract_clean_weights(macroBC_equity)
macroBC_bond = extract_clean_weights(macroBC_bond)
macroBC_rates = extract_clean_weights(macroBC_rates)
macroBC_fx = extract_clean_weights(macroBC_fx)



### Asset Data

In [179]:
# --- Load Equity Price Data ---
equity_prices = pd.read_excel(
    "../Data_Ryan/Equity Data.xlsx",
    index_col = 0,
    parse_dates = True
)
equity_prices.index = pd.to_datetime(equity_prices.index)
equity_prices.index = equity_prices.index + pd.offsets.MonthEnd(0)

equity_returns = equity_prices.pct_change()
equity_returns

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,-0.031954,NaN,NaN,-0.069246,-0.053835,NaN,NaN
1997-11-30,0.033550,NaN,NaN,-0.071232,0.009611,NaN,NaN
1997-12-31,0.013571,NaN,NaN,-0.075596,0.029754,NaN,NaN
1998-01-31,0.008938,NaN,NaN,0.085984,0.044176,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.047850,0.034649,0.008742,0.018397,0.017408,0.087457,0.024923
2025-07-31,0.019268,-0.002424,-0.029467,-0.011530,0.000891,0.009281,-0.026534
2025-08-31,0.015453,0.045088,0.049595,0.068686,0.032134,0.008553,0.028645


In [180]:
# --- Load Bond Futures ---
bond_futures = pd.read_excel(
    "../Data/Bond Data.xlsx",
    index_col = 0,
    parse_dates = True
)
bond_futures.index = pd.to_datetime(bond_futures.index)
bond_futures.index = bond_futures.index + pd.offsets.MonthEnd(0)

bond_returns = bond_futures.pct_change()
bond_returns

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,0.017379,0.006342,-0.000532,-0.009009,0.013870,NaN,-0.000851
1998-03-31,-0.014688,0.003317,0.002553,-0.002674,-0.009084,NaN,0.039250
1998-04-30,0.026085,0.008926,-0.001061,-0.000536,-0.013143,NaN,0.002005
1998-05-31,0.015326,0.017256,0.005684,0.005901,0.021489,NaN,-0.013159
...,...,...,...,...,...,...,...
2025-07-31,-0.032262,-0.007101,-0.022422,-0.009475,-0.015542,-0.009048,-0.044820
2025-08-31,0.023240,-0.003576,0.017618,0.012519,0.020001,0.005948,0.003818
2025-09-30,0.004901,-0.005493,0.010887,0.000139,0.021848,0.001495,0.000005


In [181]:
# --- Load Rates Futures ---
rates_futures = pd.read_excel(
    "../Data/Interest Rates Data.xlsx",
    index_col = 0,
    parse_dates = True
)
rates_futures.index = pd.to_datetime(rates_futures.index)
rates_futures.index = rates_futures.index + pd.offsets.MonthEnd(0)

rates_returns = rates_futures.pct_change()
rates_returns

,CH,EU,AU,US,EM,UK,JP
Date,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-03-31,-0.002432,NaN,0.000844,0.000524,0.013047,NaN,0.001312
1998-04-30,-0.000509,NaN,-0.000730,-0.000209,0.013644,NaN,0.000302
1998-05-31,0.000000,NaN,0.000422,-0.000733,-0.014970,NaN,0.000554
...,...,...,...,...,...,...,...
2025-07-31,0.000000,-0.001019,-0.002281,0.000000,0.000000,-0.001840,0.000000
2025-08-31,0.000000,-0.000459,-0.000207,0.000000,-0.009700,-0.002765,0.000000
2025-09-30,0.000000,0.000051,-0.001759,0.000000,-0.004021,-0.001232,0.000000


In [182]:
# --- Load Currency Prices ---
fx_data = pd.read_excel(
    "../Data/FX Data.xlsx",
    sheet_name = 'RETURNS',
    index_col = 0,
    parse_dates = True
)
fx_data.index = pd.to_datetime(fx_data.index)
fx_data.index = fx_data.index + pd.offsets.MonthEnd(0)
fx_returns = fx_data
fx_returns



,CH,EU,JP,UK,AU,EM
Date,,,,,,
1989-01-31,-0.062505,NaN,-0.044448,-0.028124,0.043550,NaN
1989-02-28,0.020885,NaN,0.025663,-0.002575,-0.095372,NaN
1989-03-31,-0.067754,NaN,-0.048472,-0.030821,0.027844,NaN
1989-04-30,-0.009532,NaN,-0.006180,0.004816,-0.025641,NaN
1989-05-31,-0.022300,NaN,-0.070709,-0.068691,-0.047106,NaN
...,...,...,...,...,...,...
2025-06-30,0.033185,0.036774,-0.003389,0.020141,0.022884,0.004021
2025-07-31,-0.027679,-0.033666,-0.048025,-0.038372,-0.024318,-0.009627
2025-08-31,0.010759,0.021636,0.021495,0.022178,0.017316,0.009796


### Riskfree Data

In [183]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    "../Data_Ryan/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
riskfree

C:\Users\Sedláček\AppData\Local\Temp\ipykernel_30000\2112351923.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

### Benchmark Data

In [184]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    "../Data/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()


benchmark_return = benchmark_data[['Full Benchmark']].pct_change()
benchmark_return = benchmark_return.squeeze()
benchmark_return

Date
1986-12-31         NaN
1987-01-31         NaN
1987-02-28         NaN
1987-03-31         NaN
1987-04-30         NaN
                ...   
2025-07-31    0.006737
2025-08-31    0.023840
2025-09-30    0.027309
2025-10-31    0.015763
2025-11-30    0.003025
Freq: ME, Name: Full Benchmark, Length: 468, dtype: float64

### Factor Data

In [185]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    "../Data/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data.dropna(inplace=True)
famafrench_data

C:\Users\Sedláček\AppData\Local\Temp\ipykernel_30000\383609709.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,UMD,BAB,RMW,CMA
DATE,,,,,,,
1980-01-31,0.0550,0.0188,0.0185,0.0745,-0.0056,-0.0184,0.0189
1980-02-29,-0.0123,-0.0162,0.0059,0.0789,0.0571,-0.0095,0.0292
1980-03-31,-0.1289,-0.0697,-0.0096,-0.0958,0.0605,0.0182,-0.0105
1980-04-30,0.0396,0.0105,0.0103,-0.0048,0.0145,-0.0218,0.0034
1980-05-31,0.0526,0.0200,0.0038,-0.0118,-0.0311,0.0043,-0.0063
...,...,...,...,...,...,...,...
2025-06-30,0.0486,-0.0002,-0.0160,-0.0264,-0.0246,-0.0320,0.0145
2025-07-31,0.0198,-0.0015,-0.0127,-0.0096,-0.0842,-0.0029,-0.0208
2025-08-31,0.0185,0.0488,0.0442,-0.0354,0.0391,-0.0068,0.0207


### Hedging Data

In [186]:
# Read the CSV
fx_predictions = pd.read_csv("../Data/FX_predictions.csv", index_col=0, parse_dates=["Date"])
fx_predictions.index = fx_predictions.index + pd.offsets.MonthEnd(0)

# Keep only Date, Region, Hedge Ratio
df_fx = fx_predictions[["Region", "Hedge_Ratio_Next_Month"]]

# Pivot to wide format
hedge_ratio = df_fx.pivot_table(
    index=df_fx.index,
    columns="Region",
    values="Hedge_Ratio_Next_Month"
)

if "GB" in hedge_ratio.columns:
    hedge_ratio = hedge_ratio.rename(columns={"GB": "UK"})

# Ensure proper column order (optional)
df_fhedge_ratiox_wide = hedge_ratio.sort_index(axis=1)

hedge_ratio.dropna(inplace=True)
hedge_ratio


Region,AU,CH,EM,EU,JP,UK
Date,,,,,,
2000-03-31,1.00,0.00,0.75,0.00,0.50,0.75
2000-04-30,1.00,0.00,0.75,0.00,0.50,0.75
2000-05-31,0.75,0.00,0.75,0.00,0.75,0.75
2000-06-30,1.00,0.00,0.75,0.00,0.75,1.00
2000-07-31,1.00,0.00,0.75,0.50,0.75,1.00
...,...,...,...,...,...,...
2025-04-30,0.50,0.00,0.00,0.00,0.00,0.00
2025-05-31,0.50,0.50,0.00,0.00,0.00,0.00
2025-06-30,0.50,0.75,0.00,0.00,0.00,0.00


In [187]:
# --- Load Currency Prices ---
hedge_return = pd.read_excel(
    "../Data_Ryan/FX Data.xlsx",
    sheet_name = 'HEDGE RETURN',
    index_col = 0,
    parse_dates = True
)
hedge_return.index = pd.to_datetime(hedge_return.index)
hedge_return.index = hedge_return.index + pd.offsets.MonthEnd(0)
hedge_returns = hedge_return
hedge_returns

,CH,EU,JP,UK,AU,EM
Date,,,,,,
1989-01-31,0.062505,NaN,0.044448,0.028124,-0.043550,NaN
1989-02-28,-0.020885,NaN,-0.025663,0.002575,0.095372,NaN
1989-03-31,0.067754,NaN,0.048472,0.030821,-0.027844,NaN
1989-04-30,0.009532,NaN,0.006180,-0.004816,0.025641,NaN
1989-05-31,0.022300,NaN,0.070709,0.068691,0.047106,NaN
...,...,...,...,...,...,...
2025-06-30,-0.033185,-0.036774,0.003389,-0.020141,-0.022884,-0.004021
2025-07-31,0.027679,0.033666,0.048025,0.038372,0.024318,0.009627
2025-08-31,-0.010759,-0.021636,-0.021495,-0.022178,-0.017316,-0.009796


## Combine Macro Strategies

In [188]:
macro_equity = combine_macro_weights(
    dfs=[
        macroRS_equity,
        macroIT_equity,
        macroMP_equity,
        macroBC_equity
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)
macro_bond = combine_macro_weights(
    dfs=[
        macroRS_bond,
        macroIT_bond,
        macroMP_bond,
        macroBC_bond
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

macro_rates = combine_macro_weights(
    dfs=[
        macroRS_rates,
        macroIT_rates,
        macroMP_rates,
        macroBC_rates
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

macro_fx = combine_macro_weights(
    dfs=[
        macroRS_fx,
        macroIT_fx,
        macroMP_fx,
        macroBC_fx
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

# Combined Strategies

## 1. Trend + Macro

### Individual Backtests

In [189]:
bond_macro_trend = combine_two_weight_tables(trend_bond, macro_bond, 0.5, 0.5)

bond_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = bond_macro_trend,
    returns      = bond_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [190]:
equity_macro_trend = combine_two_weight_tables(trend_equity, macro_equity, 0.5, 0.5)

equity_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = equity_macro_trend,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [191]:
fx_macro_trend = combine_two_weight_tables(trend_fx, macro_fx, 0.5, 0.5)

fx_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = fx_macro_trend,
    returns      = fx_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [192]:
rates_macro_trend = combine_two_weight_tables(trend_rates, macro_rates, 0.5, 0.5)

rates_macro_trend_results = pmp.run_cc_strategy_drift(
    weights      = rates_macro_trend,
    returns      = rates_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

### Combined

In [193]:
macro_trend = combine_strategy_results(
    dfs=[
        equity_macro_trend_results,
        bond_macro_trend_results,
        fx_macro_trend_results,
        rates_macro_trend_results
    ],
    weights=[0.25, 0.25, 0.25, 0.25]
)

macro_trend

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
1999-12-31,0.002598,0.002830,0.000000,1.160779,0.000232,0.0044,0.297837,-0.273488,0.136485,-0.135733,0.239580,0.049964,-0.311815
2000-01-31,0.006800,0.007015,0.000000,1.075494,0.000215,0.0041,-0.314096,-0.279484,0.272244,0.001191,0.237705,0.039860,0.049595
2000-02-29,0.000681,0.000829,-0.001213,0.740275,0.000148,0.0043,-0.167407,0.147284,0.259938,-0.001029,0.200007,-0.127635,-0.310328
2000-03-31,-0.001545,-0.001420,0.054429,0.622418,0.000124,0.0047,-0.102507,0.036460,0.175768,-0.254146,0.423009,-0.025491,-0.254513
2000-04-30,0.005603,0.005671,-0.038301,0.340627,0.000068,0.0046,-0.185875,0.044154,0.243049,-0.109221,0.243613,-0.080584,-0.149464
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.000375,0.000513,0.038273,0.693248,0.000139,0.0034,-0.201900,0.160602,0.234404,-0.260864,-0.131435,0.079834,0.119873
2025-07-31,-0.006274,-0.006014,0.006737,1.297450,0.000259,0.0034,-0.042306,-0.147099,-0.130201,0.287470,-0.068989,0.181379,-0.086267
2025-08-31,-0.000426,-0.000276,0.023840,0.750896,0.000150,0.0038,0.037803,0.030047,0.248381,-0.001536,-0.199258,0.117242,-0.232955


### Hedged Performance

In [194]:
macro_trend_hedged = apply_fx_hedging(
    backtest_df=macro_trend,
    hedge_ratio=hedge_ratio,
    hedge_returns=hedge_returns
)

macro_trend_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
1999-12-31,NaN,0.002830,0.000000,1.160779,0.000232,0.0044,0.297837,-0.273488,0.136485,-0.135733,0.239580,0.049964,-0.311815
2000-01-31,NaN,0.007015,0.000000,1.075494,0.000215,0.0041,-0.314096,-0.279484,0.272244,0.001191,0.237705,0.039860,0.049595
2000-02-29,NaN,0.000829,-0.001213,0.740275,0.000148,0.0043,-0.167407,0.147284,0.259938,-0.001029,0.200007,-0.127635,-0.310328
2000-03-31,NaN,-0.001420,0.054429,0.622418,0.000124,0.0047,-0.102507,0.036460,0.175768,-0.254146,0.423009,-0.025491,-0.254513
2000-04-30,NaN,0.005671,-0.038301,0.340627,0.000068,0.0046,-0.185875,0.044154,0.243049,-0.109221,0.243613,-0.080584,-0.149464
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.001312,0.000513,0.038273,0.693248,0.000139,0.0034,-0.201900,0.160602,0.234404,-0.260864,-0.131435,0.079834,0.119873
2025-07-31,-0.005003,-0.006014,0.006737,1.297450,0.000259,0.0034,-0.042306,-0.147099,-0.130201,0.287470,-0.068989,0.181379,-0.086267
2025-08-31,-0.002199,-0.000276,0.023840,0.750896,0.000150,0.0038,0.037803,0.030047,0.248381,-0.001536,-0.199258,0.117242,-0.232955


In [195]:
macro_trend_hedged.dropna(inplace=True)
macro_trend_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2010-09-30,0.006134,-0.027009,0.054015,0.388695,0.000078,0.0001,-0.311098,0.326658,-0.047686,-0.308829,0.276839,-0.125376,0.162483
2010-10-31,0.003274,0.002010,0.024512,0.676573,0.000135,0.0001,-0.152404,0.175608,0.028609,-0.298988,0.257299,0.008973,-0.017087
2010-11-30,0.009470,0.007616,-0.035641,0.581369,0.000116,0.0001,-0.101900,0.316582,0.074113,-0.031399,-0.169873,-0.018458,-0.061450
2010-12-31,0.002986,0.012823,0.043537,0.692530,0.000139,0.0001,0.147585,0.020013,0.133213,-0.299835,0.107878,-0.034993,-0.061038
2011-01-31,-0.005139,-0.005468,0.010744,0.519236,0.000104,0.0001,0.000261,0.049441,0.167145,-0.250260,-0.121496,-0.039566,0.189007
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30,-0.011860,-0.011941,0.014553,0.809146,0.000162,0.0035,-0.019762,-0.227466,0.270484,-0.264928,0.059560,0.108221,0.061949
2025-05-31,0.003561,0.003009,0.045232,0.430671,0.000086,0.0038,-0.297310,-0.081391,0.258070,-0.226773,0.002129,0.290318,0.057965
2025-06-30,-0.001312,0.000513,0.038273,0.693248,0.000139,0.0034,-0.201900,0.160602,0.234404,-0.260864,-0.131435,0.079834,0.119873


In [196]:
macro_trend_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(macro_trend_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,7.8398,1.1700
Arithm Avg Xs Return,6.5258,-0.1440
Std Xs Returns,10.0173,1.9439
Sharpe Arithmetic,0.6515,-0.0741
Geom Avg Total Return,7.5891,1.1563
Geom Avg Xs Return,6.2686,-0.1642
Sharpe Geometric,0.6258,-0.0845
Min Xs Return,-8.5349,-1.5109
Max Xs Return,8.6715,2.0161
Skewness,-0.3701,0.4920


In [197]:
pmp.run_factor_regression(macro_trend_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,1.1700
Arithm Avg Xs Return,-0.1440
Std Xs Returns,1.9439
Sharpe Arithmetic,-0.0741
Geom Avg Total Return,1.1563
Geom Avg Xs Return,-0.1642
Sharpe Geometric,-0.0845
Min Xs Return,-1.5109
Max Xs Return,2.0161
Skewness,0.4920


### Unhedged Performance

In [198]:
macro_trend.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(macro_trend, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,6.2707,2.1487
Arithm Avg Xs Return,4.4143,0.2924
Std Xs Returns,9.7797,2.4271
Sharpe Arithmetic,0.4514,0.1205
Geom Avg Total Return,5.9498,2.1383
Geom Avg Xs Return,4.0792,0.2677
Sharpe Geometric,0.4171,0.1103
Min Xs Return,-10.2114,-3.0546
Max Xs Return,8.6715,3.2044
Skewness,-0.4281,-0.1221


In [199]:
pmp.run_factor_regression(macro_trend, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.1487
Arithm Avg Xs Return,0.2924
Std Xs Returns,2.4271
Sharpe Arithmetic,0.1205
Geom Avg Total Return,2.1383
Geom Avg Xs Return,0.2677
Sharpe Geometric,0.1103
Min Xs Return,-3.0546
Max Xs Return,3.2044
Skewness,-0.1221


## 2.  Trend + Fundamental

### Individual Backtests

In [200]:
fundamental_equity_results = pmp.run_cc_strategy_drift(
    weights      = fundamental_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [201]:
trend_equity_results = pmp.run_cc_strategy_drift(
    weights      = trend_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [202]:
trend_bond_results = pmp.run_cc_strategy_drift(
    weights      = trend_bond,
    returns      = bond_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [203]:
trend_fx_results = pmp.run_cc_strategy_drift(
    weights      = trend_fx,
    returns      = fx_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [204]:
trend_rates_results = pmp.run_cc_strategy_drift(
    weights      = trend_rates,
    returns      = rates_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

### Combined

In [205]:
fundamental_trend = combine_strategy_results(
    dfs=[
        fundamental_equity_results,
        trend_equity_results,
        trend_bond_results,
        trend_fx_results,
        trend_rates_results,
    ],
    weights=[0.50, 0.125, 0.125, 0.125, 0.125]
)

fundamental_trend

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,0.004489,0.004670,-0.028188,0.906972,0.000181,0.0051,-0.048074,0.148026,0.243186,-0.315472,-0.012485,-0.427301,0.255231
2000-12-31,0.004540,0.004729,0.024349,0.941222,0.000188,0.0050,-0.119313,0.064565,0.189066,-0.220341,-0.285759,-0.084215,0.262252
2001-01-31,0.002593,0.002789,0.010556,0.980868,0.000196,0.0054,0.272153,-0.309289,0.294974,-0.211998,0.045142,0.185220,-0.129287
2001-02-28,-0.009308,-0.009212,-0.048796,0.480563,0.000096,0.0038,0.260085,-0.197897,0.264430,-0.102347,-0.089181,0.048037,0.041240
2001-03-31,-0.001991,-0.001879,-0.049222,0.560200,0.000112,0.0042,0.210390,-0.067332,0.061097,-0.064931,-0.304210,0.182995,0.029002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.005725,-0.005542,0.038273,0.917528,0.000184,0.0034,-0.154698,0.029568,-0.099433,-0.384070,-0.099627,0.177522,0.581890
2025-07-31,0.010211,0.010516,0.006737,1.523416,0.000305,0.0034,0.086345,-0.258655,-0.410137,0.116037,-0.232753,0.303826,0.407658
2025-08-31,0.008178,0.008431,0.023840,1.263619,0.000253,0.0038,0.110933,0.075497,0.055265,-0.111806,-0.147422,0.246240,-0.182681


### Hedged Performance

In [206]:
fundamental_trend_hedged = apply_fx_hedging(
    backtest_df=fundamental_trend,
    hedge_ratio=hedge_ratio,
    hedge_returns=hedge_returns
)


In [207]:
fundamental_trend_hedged.dropna(inplace=True)
fundamental_trend_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2010-09-30,0.017162,-0.022268,0.054015,0.583997,0.000117,0.0001,-0.413634,0.436034,-0.047686,-0.365683,0.198301,0.095014,0.187979
2010-10-31,0.013204,0.014365,0.024512,0.932287,0.000186,0.0001,-0.222378,0.146724,0.028609,-0.226681,0.245364,0.319597,-0.063399
2010-11-30,0.017572,0.013825,-0.035641,1.093945,0.000219,0.0001,-0.107575,0.515864,0.074113,-0.087032,-0.033124,-0.100275,-0.061844
2010-12-31,0.015548,0.030928,0.043537,0.967353,0.000193,0.0001,0.243814,0.029117,0.133213,-0.358900,0.421724,-0.191938,-0.015517
2011-01-31,-0.018466,-0.020800,0.010744,0.706962,0.000141,0.0001,0.130476,0.041710,0.167145,-0.387582,0.143994,-0.117310,0.109832
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30,-0.002677,-0.001340,0.014553,1.362580,0.000273,0.0035,0.086651,-0.248155,-0.072926,-0.354300,0.245155,0.318842,0.049277
2025-05-31,0.018058,0.017958,0.045232,1.014355,0.000203,0.0038,-0.122904,-0.290202,-0.005322,-0.329256,-0.056853,0.323044,0.472850
2025-06-30,-0.004691,-0.005542,0.038273,0.917528,0.000184,0.0034,-0.154698,0.029568,-0.099433,-0.384070,-0.099627,0.177522,0.581890


In [208]:
fundamental_trend_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_trend_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,7.8398,1.6186
Arithm Avg Xs Return,6.5258,0.3046
Std Xs Returns,10.0173,5.6460
Sharpe Arithmetic,0.6515,0.0539
Geom Avg Total Return,7.5891,1.4695
Geom Avg Xs Return,6.2686,0.1489
Sharpe Geometric,0.6258,0.0264
Min Xs Return,-8.5349,-4.6011
Max Xs Return,8.6715,4.3528
Skewness,-0.3701,-0.1378


In [209]:
pmp.run_factor_regression(fundamental_trend_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,1.6186
Arithm Avg Xs Return,0.3046
Std Xs Returns,5.6460
Sharpe Arithmetic,0.0539
Geom Avg Total Return,1.4695
Geom Avg Xs Return,0.1489
Sharpe Geometric,0.0264
Min Xs Return,-4.6011
Max Xs Return,4.3528
Skewness,-0.1378


### Unhedged Performance

In [210]:
fundamental_trend.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_trend, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,6.5885,2.9331
Arithm Avg Xs Return,4.8705,1.2151
Std Xs Returns,9.7763,7.0417
Sharpe Arithmetic,0.4982,0.1726
Geom Avg Total Return,6.2840,2.7021
Geom Avg Xs Return,4.5539,0.9720
Sharpe Geometric,0.4658,0.1380
Min Xs Return,-10.2114,-23.1376
Max Xs Return,8.6715,5.9557
Skewness,-0.4663,-4.7675


In [211]:
pmp.run_factor_regression(fundamental_trend, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.9331
Arithm Avg Xs Return,1.2151
Std Xs Returns,7.0417
Sharpe Arithmetic,0.1726
Geom Avg Total Return,2.7021
Geom Avg Xs Return,0.9720
Sharpe Geometric,0.1380
Min Xs Return,-23.1376
Max Xs Return,5.9557
Skewness,-4.7675


## 3. Fundamental + Macro

### Individual Backtests

In [212]:
fundamental_equity_results = pmp.run_cc_strategy_drift(
    weights      = fundamental_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [213]:
macro_equity_results = pmp.run_cc_strategy_drift(
    weights      = macro_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [214]:
macro_bond_results = pmp.run_cc_strategy_drift(
    weights      = macro_bond,
    returns      = bond_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [215]:
macro_fx_results = pmp.run_cc_strategy_drift(
    weights      = macro_fx,
    returns      = fx_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [216]:
macro_rates_results = pmp.run_cc_strategy_drift(
    weights      = macro_rates,
    returns      = rates_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

### Combined

In [217]:
fundamental_macro = combine_strategy_results(
    dfs=[
        fundamental_equity_results,
        macro_equity_results,
        macro_bond_results,
        macro_fx_results,
        macro_rates_results,
    ],
    weights=[0.50, 0.125, 0.125, 0.125, 0.125]
)

fundamental_macro

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2002-03-31,0.003593,0.003635,0.022050,0.208895,0.000042,0.0013,0.000000,0.054905,0.000000,0.075364,0.293798,-0.351011,-0.175822
2002-04-30,0.016605,0.016657,-0.001331,0.256592,0.000051,0.0015,0.000000,0.193862,0.000000,0.007466,0.217819,-0.269533,-0.030262
2002-05-31,0.027636,0.027683,0.014872,0.232319,0.000046,0.0014,0.000000,0.053074,0.000000,0.007090,0.354032,-0.104924,-0.075821
2002-06-30,-0.007755,-0.007701,-0.007289,0.270819,0.000054,0.0013,0.000000,-0.055004,0.000000,-0.028866,0.148182,-0.052697,0.065387
2002-07-31,-0.018506,-0.018419,-0.035635,0.434745,0.000087,0.0015,0.000000,-0.040038,0.000000,0.018298,-0.199947,0.365980,0.057062
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-31,0.014832,0.014949,0.045232,0.583684,0.000117,0.0038,0.174406,-0.208811,-0.263392,-0.102483,-0.058982,0.032725,0.414885
2025-06-30,-0.006100,-0.006055,0.038273,0.224280,0.000045,0.0034,0.047202,-0.131034,-0.333837,-0.123206,0.031808,0.097688,0.462018
2025-07-31,0.016484,0.016530,0.006737,0.225966,0.000045,0.0034,0.128651,-0.111556,-0.279936,-0.171433,-0.163764,0.122448,0.493925


### Hedged Performance

In [237]:
fundamental_macro_hedged = apply_fx_hedging(
    backtest_df=fundamental_macro,
    hedge_ratio=hedge_ratio,
    hedge_returns=hedge_returns
)


In [235]:
fundamental_macro_hedged.dropna(inplace=True)
fundamental_macro_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2010-09-30,0.011028,0.004742,0.054015,0.195302,0.000039,0.0001,-0.102536,0.109376,0.000000,-0.056854,-0.078538,0.220391,0.025496
2010-10-31,0.009930,0.012355,0.024512,0.255714,0.000051,0.0001,-0.069974,-0.028883,0.000000,0.072307,-0.011935,0.310624,-0.046312
2010-11-30,0.008102,0.006209,-0.035641,0.512575,0.000103,0.0001,-0.005675,0.199282,0.000000,-0.055633,0.136749,-0.081817,-0.000394
2010-12-31,0.012562,0.018105,0.043537,0.274822,0.000055,0.0001,0.096229,0.009104,0.000000,-0.059064,0.313846,-0.156945,0.045521
2011-01-31,-0.013327,-0.015333,0.010744,0.187726,0.000038,0.0001,0.130215,-0.007731,0.000000,-0.137321,0.265490,-0.077744,-0.079176
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30,0.009183,0.010601,0.014553,0.553434,0.000111,0.0035,0.106413,-0.020689,-0.343411,-0.089372,0.185595,0.210620,-0.012672
2025-05-31,0.014497,0.014949,0.045232,0.583684,0.000117,0.0038,0.174406,-0.208811,-0.263392,-0.102483,-0.058982,0.032725,0.414885
2025-06-30,-0.003379,-0.006055,0.038273,0.224280,0.000045,0.0034,0.047202,-0.131034,-0.333837,-0.123206,0.031808,0.097688,0.462018


In [ ]:
fundamental_macro_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_macro_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,7.8398,1.7626
Arithm Avg Xs Return,6.5258,0.4486
Std Xs Returns,10.0173,5.0381
Sharpe Arithmetic,0.6515,0.0890
Geom Avg Total Return,7.5891,1.6481
Geom Avg Xs Return,6.2686,0.3275
Sharpe Geometric,0.6258,0.0650
Min Xs Return,-8.5349,-4.2473
Max Xs Return,8.6715,3.9448
Skewness,-0.3701,-0.1013


In [238]:
print(fundamental_macro_hedged)

             ret_net  ret_gross    ret_bm  turnover     tcost  ret_rf  \
Date                                                                    
2002-03-31       NaN   0.003635  0.022050  0.208895  0.000042  0.0013   
2002-04-30       NaN   0.016657 -0.001331  0.256592  0.000051  0.0015   
2002-05-31       NaN   0.027683  0.014872  0.232319  0.000046  0.0014   
2002-06-30       NaN  -0.007701 -0.007289  0.270819  0.000054  0.0013   
2002-07-31       NaN  -0.018419 -0.035635  0.434745  0.000087  0.0015   
...              ...        ...       ...       ...       ...     ...   
2025-05-31  0.014497   0.014949  0.045232  0.583684  0.000117  0.0038   
2025-06-30 -0.003379  -0.006055  0.038273  0.224280  0.000045  0.0034   
2025-07-31  0.012847   0.016530  0.006737  0.225966  0.000045  0.0034   
2025-08-31  0.008386   0.008707  0.023840  0.512723  0.000103  0.0038   
2025-09-30       NaN  -0.002562  0.027309  0.672767  0.000135  0.0033   

                w_AU      w_CH      w_EM      w_EU

In [221]:
pmp.run_factor_regression(fundamental_macro_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,1.7626
Arithm Avg Xs Return,0.4486
Std Xs Returns,5.0381
Sharpe Arithmetic,0.0890
Geom Avg Total Return,1.6481
Geom Avg Xs Return,0.3275
Sharpe Geometric,0.0650
Min Xs Return,-4.2473
Max Xs Return,3.9448
Skewness,-0.1013


### Unhedged Performance

In [222]:
fundamental_macro.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(fundamental_macro, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,7.4649,3.4476
Arithm Avg Xs Return,5.8731,1.8558
Std Xs Returns,9.7516,4.6430
Sharpe Arithmetic,0.6023,0.3997
Geom Avg Total Return,7.2155,3.3920
Geom Avg Xs Return,5.6134,1.7899
Sharpe Geometric,0.5756,0.3855
Min Xs Return,-10.2114,-3.2348
Max Xs Return,8.6715,4.2548
Skewness,-0.4969,0.2226


In [223]:
pmp.run_factor_regression(fundamental_macro, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,3.4476
Arithm Avg Xs Return,1.8558
Std Xs Returns,4.6430
Sharpe Arithmetic,0.3997
Geom Avg Total Return,3.3920
Geom Avg Xs Return,1.7899
Sharpe Geometric,0.3855
Min Xs Return,-3.2348
Max Xs Return,4.2548
Skewness,0.2226


## 4. Fundamental + Macro + Trend

### Individual Backtests

#### Macro + Trend (Weight = 2/3)

In [224]:
macro_trend # see above

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
1999-12-31,0.002598,0.002830,0.000000,1.160779,0.000232,0.0044,0.297837,-0.273488,0.136485,-0.135733,0.239580,0.049964,-0.311815
2000-01-31,0.006800,0.007015,0.000000,1.075494,0.000215,0.0041,-0.314096,-0.279484,0.272244,0.001191,0.237705,0.039860,0.049595
2000-02-29,0.000681,0.000829,-0.001213,0.740275,0.000148,0.0043,-0.167407,0.147284,0.259938,-0.001029,0.200007,-0.127635,-0.310328
2000-03-31,-0.001545,-0.001420,0.054429,0.622418,0.000124,0.0047,-0.102507,0.036460,0.175768,-0.254146,0.423009,-0.025491,-0.254513
2000-04-30,0.005603,0.005671,-0.038301,0.340627,0.000068,0.0046,-0.185875,0.044154,0.243049,-0.109221,0.243613,-0.080584,-0.149464
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.000375,0.000513,0.038273,0.693248,0.000139,0.0034,-0.201900,0.160602,0.234404,-0.260864,-0.131435,0.079834,0.119873
2025-07-31,-0.006274,-0.006014,0.006737,1.297450,0.000259,0.0034,-0.042306,-0.147099,-0.130201,0.287470,-0.068989,0.181379,-0.086267
2025-08-31,-0.000426,-0.000276,0.023840,0.750896,0.000150,0.0038,0.037803,0.030047,0.248381,-0.001536,-0.199258,0.117242,-0.232955


#### Fundamental (Weight = 1/3)

In [225]:
fundamental_equity_results

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_UK,w_JP,w_US
Date,,,,,,,,,,,,,
2000-11-30,0.027872,0.027872,-0.028188,0.000000,0.000000,0.0051,0.000000,0.000000,0.000000,0.000000,-0.624071,0.151240,0.177581
2000-12-31,0.018812,0.018919,0.024349,0.534371,0.000107,0.0050,0.000000,0.000000,0.000000,0.000000,-0.147962,-0.255471,0.025404
2001-01-31,-0.004175,-0.004093,0.010556,0.408307,0.000082,0.0054,0.000000,0.000000,0.000000,0.000000,0.311027,0.024310,-0.051180
2001-02-28,-0.030311,-0.030293,-0.048796,0.091500,0.000018,0.0038,0.000000,0.000000,0.000000,0.000000,0.309513,0.144530,-0.017178
2001-03-31,-0.006545,-0.006501,-0.049222,0.219393,0.000044,0.0042,0.000000,0.000000,0.000000,0.000000,0.335054,0.002969,-0.246744
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.012200,-0.012110,0.038273,0.448561,0.000090,0.0034,0.094403,-0.262069,-0.667673,-0.246411,0.195375,0.063617,0.924035
2025-07-31,0.032969,0.033059,0.006737,0.451932,0.000090,0.0034,0.257303,-0.223111,-0.559872,-0.342865,0.244895,-0.327527,0.987850
2025-08-31,0.017209,0.017414,0.023840,1.025445,0.000205,0.0038,0.146260,0.090900,-0.386231,-0.220539,0.257997,0.103672,0.100548


### Combined

In [226]:
combined = combine_strategy_results(
    dfs=[
        fundamental_equity_results,
        macro_trend
    ],
    weights=[0.25, 0.75]
)

combined

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,-0.000117,0.000019,-0.028188,0.680229,0.000136,0.0051,-0.036056,0.111019,0.182390,-0.236604,-0.028269,-0.242467,0.169225
2000-12-31,0.001054,0.001182,0.024349,0.639120,0.000128,0.0050,-0.089485,0.048423,0.141800,-0.165256,-0.182385,-0.044666,0.193513
2001-01-31,0.002467,0.002604,0.010556,0.684613,0.000137,0.0054,0.204115,-0.231967,0.221231,-0.158999,0.030817,0.100036,-0.090568
2001-02-28,-0.003192,-0.003122,-0.048796,0.348985,0.000070,0.0038,0.195064,-0.148423,0.198322,-0.076761,-0.084952,-0.002661,0.033077
2001-03-31,-0.000675,-0.000597,-0.049222,0.392726,0.000079,0.0042,0.157792,-0.050499,0.045822,-0.048698,-0.228529,0.095365,0.052594
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.002769,-0.002642,0.038273,0.632076,0.000126,0.0034,-0.127824,0.054935,0.008884,-0.257251,-0.082672,0.108720,0.320913
2025-07-31,0.003537,0.003754,0.006737,1.086070,0.000217,0.0034,0.032596,-0.166102,-0.237619,0.129886,-0.133624,0.197258,0.182262
2025-08-31,0.003983,0.004147,0.023840,0.819533,0.000164,0.0038,0.064917,0.045261,0.089728,-0.056287,-0.123525,0.152430,-0.149579


### Hedged Performance

In [227]:
combined_hedged = apply_fx_hedging(
    backtest_df=combined,
    hedge_ratio=hedge_ratio,
    hedge_returns=hedge_returns
)


In [228]:
combined_hedged.dropna(inplace=True)
combined_hedged

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2010-09-30,0.010115,-0.017886,0.054015,0.389172,0.000078,0.0001,-0.284591,0.299681,-0.035764,-0.260049,0.168360,0.016163,0.134610
2010-10-31,0.007420,0.007685,0.024512,0.635287,0.000127,0.0001,-0.149290,0.117264,0.021456,-0.188088,0.187007,0.162042,-0.035971
2010-11-30,0.011154,0.008817,-0.035641,0.692315,0.000138,0.0001,-0.079263,0.337077,0.055585,-0.051366,-0.059030,-0.054752,-0.046285
2010-12-31,0.008520,0.018670,0.043537,0.656809,0.000131,0.0001,0.158804,0.019562,0.099910,-0.254409,0.237832,-0.104717,-0.023018
2011-01-31,-0.010518,-0.011767,0.010744,0.483290,0.000097,0.0001,0.065303,0.033215,0.125358,-0.256356,0.041623,-0.068546,0.102168
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-30,-0.004303,-0.003655,0.014553,0.883577,0.000177,0.0035,0.038385,-0.180944,0.031158,-0.243382,0.137467,0.186476,0.040126
2025-05-31,0.009919,0.009731,0.045232,0.614845,0.000123,0.0038,-0.135779,-0.165449,0.061857,-0.221321,-0.027894,0.234101,0.250916
2025-06-30,-0.002674,-0.002642,0.038273,0.632076,0.000126,0.0034,-0.127824,0.054935,0.008884,-0.257251,-0.082672,0.108720,0.320913


In [229]:
combined_hedged.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(combined_hedged, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,7.8398,1.4303
Arithm Avg Xs Return,6.5258,0.1163
Std Xs Returns,10.0173,3.0805
Sharpe Arithmetic,0.6515,0.0377
Geom Avg Total Return,7.5891,1.3912
Geom Avg Xs Return,6.2686,0.0707
Sharpe Geometric,0.6258,0.0229
Min Xs Return,-8.5349,-2.3890
Max Xs Return,8.6715,2.2784
Skewness,-0.3701,-0.0788


In [230]:
pmp.run_factor_regression(combined_hedged, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,1.4303
Arithm Avg Xs Return,0.1163
Std Xs Returns,3.0805
Sharpe Arithmetic,0.0377
Geom Avg Total Return,1.3912
Geom Avg Xs Return,0.0707
Sharpe Geometric,0.0229
Min Xs Return,-2.3890
Max Xs Return,2.2784
Skewness,-0.0788


### Unhedged Performance

In [231]:
combined.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(combined, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,6.5885,2.3796
Arithm Avg Xs Return,4.8705,0.6616
Std Xs Returns,9.7763,3.8801
Sharpe Arithmetic,0.4982,0.1705
Geom Avg Total Return,6.2840,2.3267
Geom Avg Xs Return,4.5539,0.5966
Sharpe Geometric,0.4658,0.1538
Min Xs Return,-10.2114,-12.3325
Max Xs Return,8.6715,3.4340
Skewness,-0.4663,-4.2515


In [232]:
pmp.run_factor_regression(combined, famafrench_data, alreadyXs=True)

,Strategy
Arithm Avg Total Return,2.3796
Arithm Avg Xs Return,0.6616
Std Xs Returns,3.8801
Sharpe Arithmetic,0.1705
Geom Avg Total Return,2.3267
Geom Avg Xs Return,0.5966
Sharpe Geometric,0.1538
Min Xs Return,-12.3325
Max Xs Return,3.4340
Skewness,-4.2515
